In [1]:
import sys
import os
sys.path.append('../')

In [2]:
from pathlib import Path

import gym_pusht  # noqa: F401
import gymnasium as gym
import imageio
import numpy
import torch
from huggingface_hub import snapshot_download

from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

/home/sm/anaconda3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create a directory to store the video of the evaluation

In [3]:
output_directory = Path("outputs/eval/example_pusht_diffusion")
output_directory.mkdir(parents=True, exist_ok=True)

### Download the diffusion policy for pusht environment

In [4]:
pretrained_policy_path = Path(snapshot_download("lerobot/diffusion_pusht"))

policy = DiffusionPolicy.from_pretrained(pretrained_policy_path)
policy.eval()

Fetching 11 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 244112.93it/s]


Loading weights from local directory


DiffusionPolicy(
  (normalize_inputs): Normalize(
    (buffer_observation_image): ParameterDict(
        (mean): Parameter containing: [torch.FloatTensor of size 3x1x1]
        (std): Parameter containing: [torch.FloatTensor of size 3x1x1]
    )
    (buffer_observation_state): ParameterDict(
        (max): Parameter containing: [torch.FloatTensor of size 2]
        (min): Parameter containing: [torch.FloatTensor of size 2]
    )
  )
  (normalize_targets): Normalize(
    (buffer_action): ParameterDict(
        (max): Parameter containing: [torch.FloatTensor of size 2]
        (min): Parameter containing: [torch.FloatTensor of size 2]
    )
  )
  (unnormalize_outputs): Unnormalize(
    (buffer_action): ParameterDict(
        (max): Parameter containing: [torch.FloatTensor of size 2]
        (min): Parameter containing: [torch.FloatTensor of size 2]
    )
  )
  (diffusion): DiffusionModel(
    (rgb_encoder): DiffusionRgbEncoder(
      (center_crop): CenterCrop(size=[84, 84])
      (maybe_

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Device set to:", device)
else:
    device = torch.device("cpu")
    print(f"GPU is not available. Device set to: {device}. Inference will be slower than on GPU.")
    # Decrease the number of reverse-diffusion steps (trades off a bit of quality for 10x speed)
    policy.diffusion.num_inference_steps = 10

GPU is available. Device set to: cuda


In [6]:
policy.to(device)

DiffusionPolicy(
  (normalize_inputs): Normalize(
    (buffer_observation_image): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
    )
    (buffer_observation_state): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
    )
  )
  (normalize_targets): Normalize(
    (buffer_action): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
    )
  )
  (unnormalize_outputs): Unnormalize(
    (buffer_action): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
    )
  )
  (diffusion): Diff

Initialize evaluation environment to render two observation types: an image of the scene and state/position of the agent. 

The environment also automatically stops running after 300 interactions/steps.

In [7]:
env = gym.make(
    "gym_pusht/PushT-v0",
    obs_type="pixels_agent_pos",
    max_episode_steps=300,
)


In [8]:
policy.reset()
numpy_observation, info = env.reset(seed=42)

rewards = []
frames = []

step = 0
done = False
while not done:
    frames.append(env.render())
    # Prepare observation for the policy running in Pytorch
    state = torch.from_numpy(numpy_observation["agent_pos"])
    image = torch.from_numpy(numpy_observation["pixels"])

    # Convert to float32 with image from channel first in [0,255]
    # to channel last in [0,1]
    state = state.to(torch.float32)
    image = image.to(torch.float32) / 255
    image = image.permute(2, 0, 1)

    # Send data tensors from CPU to GPU
    state = state.to(device, non_blocking=True)
    image = image.to(device, non_blocking=True)

    # Add extra (empty) batch dimension, required to forward the policy
    state = state.unsqueeze(0)
    image = image.unsqueeze(0)

    # Create the policy input dictionary
    observation = {
        "observation.state": state,
        "observation.image": image,
    }

    # Predict the next action with respect to the current observation
    with torch.inference_mode():
        action = policy.select_action(observation)

    # Prepare the action for the environment
    numpy_action = action.squeeze(0).to("cpu").numpy()

    # Step through the environment and receive a new observation
    numpy_observation, reward, terminated, truncated, info = env.step(numpy_action)
    print(f"{step=} {reward=} {terminated=}")

    # Keep track of all the rewards and frames
    rewards.append(reward)
    # frames.append(env.render())

    # The rollout is considered done when the success state is reach (i.e. terminated is True),
    # or the maximum number of iterations is reached (i.e. truncated is True)
    done = terminated | truncated | done
    step += 1

if terminated:
    print("Success!")
else:
    print("Failure!")

# Get the speed of environment (i.e. its number of frames per second).
fps = env.metadata["render_fps"]

# Encode all frames into a mp4 video.
video_path = output_directory / "rollout.mp4"
imageio.mimsave(str(video_path), numpy.stack(frames), fps=fps)

print(f"Video of the evaluation is available in '{video_path}'.")

step=0 reward=0.0 terminated=False
step=1 reward=0.0 terminated=False
step=2 reward=0.0 terminated=False
step=3 reward=0.0 terminated=False
step=4 reward=0.0 terminated=False
step=5 reward=0.0 terminated=False
step=6 reward=0.0 terminated=False
step=7 reward=0.0 terminated=False
step=8 reward=0.0 terminated=False
step=9 reward=0.0 terminated=False
step=10 reward=0.0 terminated=False
step=11 reward=0.0 terminated=False
step=12 reward=0.0 terminated=False
step=13 reward=0.0 terminated=False
step=14 reward=0.0 terminated=False
step=15 reward=0.0 terminated=False
step=16 reward=0.0 terminated=False
step=17 reward=0.0 terminated=False
step=18 reward=0.0 terminated=False
step=19 reward=0.0 terminated=False
step=20 reward=0.0 terminated=False
step=21 reward=0.0 terminated=False
step=22 reward=0.0 terminated=False
step=23 reward=0.0 terminated=False
step=24 reward=0.0 terminated=False
step=25 reward=0.0 terminated=False
step=26 reward=0.0 terminated=False
step=27 reward=0.0 terminated=False
st

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


step=144 reward=0.9928034356354293 terminated=False
step=145 reward=0.9928034356354293 terminated=False
step=146 reward=0.9928034356354293 terminated=False
step=147 reward=0.9928034356354293 terminated=False
step=148 reward=1.0 terminated=True
Success!


[swscaler @ 0x5db16c0] Warning: data is not aligned! This can lead to a speed loss


Video of the evaluation is available in 'outputs/eval/example_pusht_diffusion/rollout.mp4'.


In [9]:
from IPython.display import Video

# 비디오 표시
Video(video_path, embed=True, width=640, height=360)